In [1]:
import os
import time

import numpy as np
import tensorflow as tf

In [2]:
path_to_file = tf.keras.utils.get_file(
    "shakespeare.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt",
)

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
text = open(path_to_file, "rb").read().decode(encoding="utf-8")
print(f"Length of text; {len(text)} characters")

Length of text; 1115394 characters


In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
vocab = sorted(set(text))
print(f"{len(vocab)} unique characters")

65 unique characters


In [7]:
example_texts = ["abcdefg", "xyz"]

chars = tf.strings.unicode_split(example_texts, input_encoding="UTF-8")
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [9]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None
)

In [12]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [13]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None
)

In [15]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [16]:
tf.strings.reduce_join(chars, axis=1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [17]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [18]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, "UTF-8"))
all_ids


<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1], dtype=int64)>

In [19]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [22]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode("UTF-8"))

F
i
r
s
t
 
C
i
t
i


In [26]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

In [28]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq).numpy())

[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' ']


In [31]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [32]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [33]:
dataset = sequences.map(split_input_target)

In [47]:
for input_example, target_example_batch in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target :", text_from_ids(target_example).numpy())

Input : [b" the boar did raze his helm;\nBut I disdain'd it, and did scorn to fly:\nThree times to-day my foot-cl"
 b'arkness breaks within the east.\nIn brief,--for so the season bids us be,--\nPrepare thy battle early '
 b'sighs and weeps:\nWe took this mattock and this spade from him,\nAs he was coming from this churchyard'
 b'the world either malice or\nmatter to alter it. You have an unspeakable\ncomfort of your young prince '
 b'e to my soul,\nMy soul the father; and these two beget\nA generation of still-breeding thoughts,\nAnd t'
 b' honour; and so, I pray, go with us.\n\nVIRGILIA:\nGive me excuse, good madam; I will obey you in every'
 b'hich then blew bitterly against our faces,\nAwaked the sleeping rheum, and so by chance\nDid grace our'
 b'cost ten thousand lives this day.\n3 KING HENRY VI\n\nWARWICK:\nForspent with toil, as runners with a ra'
 b"in place.\n\nPost:\n'Tell him,' quoth she, 'my mourning weeds are done,\nAnd I am ready to put armour on"
 b'to pity him, lest th

In [48]:
BATCH_SIZE=64

BUFFER_SIZE=10000

dataset = (
    dataset.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 64, 100), dtype=tf.int64, name=None))>

In [49]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [50]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        # TODO - Create an embedding layer
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        # TODO - Create a GRU layer
        self.gru = tf.keras.layers.GRU(
            rnn_units, return_sequences=True, return_state=True
        )
        # TODO - Finally connect it with a dense layer
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)
        # since we are training a text generation model,
        # we use the previous state, in training. If there is no state,
        # then we initialize the state
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [51]:
model = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
)

In [52]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [54]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print(
    "Prediction shape: ",
    example_batch_predictions.shape,
    " # (batch_size, sequence_length, vocab_size)",
)
print("Mean loss:        ", example_batch_mean_loss)

NameError: name 'example_batch_predictions' is not defined

In [ ]:
tf.exp(example_batch_mean_loss).numpy()

In [ ]:
model.compile(optimizer="adam", loss=loss)